In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)

## Summary for next time I look at the data
1. need 41697600 data
2. how do I align all data: key should be 'row_id'

#### Note: I name the dataframe as train here because I don't want to change the code below... I am just lazy

In [3]:
%%time
train = pd.read_csv('test_joined.csv', parse_dates=['timestamp'])

Wall time: 1min 11s


## Take out cloud_coverage, precip_depth_1_hr, wind_direction, and  sea_level_pressure for now

In [4]:
train = train.drop(['year_built', 'floor_count', 'Unnamed: 0', 
                    'precip_depth_1_hr', 'cloud_coverage', 'sea_level_pressure', 'wind_direction'], axis=1)
# train = train.dropna()
train.isnull().sum()/train.shape[0]

row_id             0.000000
building_id        0.000000
meter              0.000000
timestamp          0.000000
site_id            0.000000
primary_use        0.000000
square_feet        0.000000
air_temperature    0.005322
dew_temperature    0.006255
wind_speed         0.007245
dtype: float64

## Prepocess
1. Label primary_use 
2. move y to the first column
3. Add month, week day, day, hour

In [5]:
le = preprocessing.LabelEncoder()
train['primary_use'] = le.fit_transform(train['primary_use'])

move = ['timestamp'] + [x for x in train.columns if x not in ['meter_reading', 'timestamp']]
train = train[move]

In [7]:
%%time
## this may take mins
train['date_month'] = train['timestamp'].map(lambda x: x.month)
train['date_day'] = train['timestamp'].map(lambda x: x.day)
train['iso_week'] = train['timestamp'].map(lambda x: x.isocalendar()[1])
train['iso_weekend'] = train['timestamp'].map(lambda x: x.isocalendar()[2])
train['hour'] = train['timestamp'].map(lambda x: x.hour)

Wall time: 6min 26s


In [8]:
train.head()

,timestamp,row_id,building_id,meter,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,date_month,date_day,iso_week,iso_weekend,hour
0,2017-01-01,0,0,0,0,0,7432,17.8,11.7,3.6,1,1,52,7,0
1,2017-01-01,1,1,0,0,0,2720,17.8,11.7,3.6,1,1,52,7,0
2,2017-01-01,2,2,0,0,0,5376,17.8,11.7,3.6,1,1,52,7,0
3,2017-01-01,3,3,0,0,0,23685,17.8,11.7,3.6,1,1,52,7,0
4,2017-01-01,4,4,0,0,0,116607,17.8,11.7,3.6,1,1,52,7,0


In [9]:
train.shape

(41697600, 15)

## Get building_list

In [10]:
building_list = train['building_id'].drop_duplicates()
len(building_list)

1449

## Run models on test data

In [23]:
%%time
prediction = pd.DataFrame()

for sample in building_list: 
    sample_df = train[train['building_id']==sample]
    X = sample_df.iloc[:,2:]
    
    bst = lgb.Booster(model_file = 'models/model{}.txt'.format(sample))
    
    prediction_on_X_train = bst.predict(X)
    prediction_on_X_train = pd.DataFrame(prediction_on_X_train, columns=['meter_reading'])
    prediction_on_X_train[prediction_on_X_train['meter_reading']<0] = 0
    prediction_on_X_train['row_id'] = sample_df.iloc[:,1].values
    prediction_on_X_train = prediction_on_X_train[['row_id', 'meter_reading']]
    
    prediction = pd.concat([prediction, prediction_on_X_train])

Wall time: 6min 41s


In [24]:
prediction.shape

(41697600, 2)

In [26]:
prediction = prediction.sort_values('row_id')
prediction.head(10)

,row_id,meter_reading
0,0,197.406651
0,1,60.788927
0,2,6.560916
0,3,475.716874
0,4,1456.573171
0,5,12.902798
0,6,187.414600
0,7,502.050283
1,8,1074.977963
0,9,465.296370


In [27]:
prediction.to_csv('submission.csv', index=False)

In [28]:
check = pd.read_csv('submission.csv')
check.head(10)

,row_id,meter_reading
0,0,197.406651
1,1,60.788927
2,2,6.560916
3,3,475.716874
4,4,1456.573171
5,5,12.902798
6,6,187.414600
7,7,502.050283
8,8,1074.977963
9,9,465.296370
